In [9]:
import numpy as np
import cv2
import os
import pywt
import pickle
from skimage.metrics import structural_similarity as ssim

In [3]:
def image_read(image):
    return cv2.imread(image, cv2.IMREAD_COLOR)

def wavelet_decomposition(image, wavelet='haar'):
    # Determine the maximum level of decompositiom
    max_level = pywt.dwt_max_level(min(image.shape), pywt.Wavelet(wavelet).dec_len)
    coeffs = pywt.wavedec2(image, wavelet, level=max_level)
    return coeffs

def calculate_wavelet_scale_invariance_score(image, wavelet_type='haar'):
    coeffs = wavelet_decomposition(image, wavelet_type)
    # Analyze the coefficients at different scales
    scores = [np.mean(np.abs(c)) for c in coeffs[1:]]  # Skipping the approximation coefficients
    # Aggregating the scores
    scale_invariance_score = np.mean(scores)
    return scale_invariance_score

In [12]:
def run_si(dataset, save_location):
    # create folder for SI scores
    dists_score_path = os.path.join(save_location, 'si_scores')
    if not os.path.exists(dists_score_path):
        os.makedirs(dists_score_path)

    # get all exemplar category subfolders
    dtd_subfolders = [f.path for f in os.scandir(dataset) if f.is_dir()]

    # final SI scores
    syn_scores = {}

    # for every category
    for texture_dir in dtd_subfolders:
        
        # get category title from path
        name = os.path.basename(os.path.normpath(texture_dir))

        # all images within a category
        category_list = os.listdir(texture_dir)

        for texture in category_list:
            # retrieve actual image
            full_path = f'{dataset}/{name}/{texture}'
            # specific texture name
            texture_name = os.path.splitext(texture)[0]
            # load exemplar as tensor
            image = image_read(full_path)

            # Check if the image is loaded properly
            if image is None:
                scale_invariance = np.nan
            else:
                scale_invariance = calculate_wavelet_scale_invariance_score(image)

            # save SI score
            syn_scores[texture_name] = scale_invariance

    # save score dicts as files
    with open(os.path.join(dists_score_path, 'individual_scores'), 'wb') as f:
        pickle.dump(syn_scores, f)

In [ ]:
run_si('./dtd_small', './')